In [62]:
import pandas as pd
import re
from llama_index.core import Document
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from transformers import BitsAndBytesConfig
from llama_index.core import VectorStoreIndex
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import Settings
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core import PromptTemplate
import torch

from typing import List
from pydantic import BaseModel

In [40]:
# class Course(BaseModel):
#     """Data model for education course."""
#     title: str
#     description: str
#     # price: int
#     # link: str

# class CourseList(BaseModel):
#     """Data model for list of education courses."""
#     courses: List[Course]

class TechList(BaseModel):
    """Data model for 2 technology stack."""
    good: List[str]
    bad: List[str]

In [3]:
def create_docs(df):
    documents = []
    for i, row in df.iterrows():
        metadata = {"ссылка": row.url, "заголовок": row.title, "цена": row.price, "инструменты": ", ".join(row.technologies)}
        documents.append(Document(text=row.description, metadata=metadata, excluded_embed_metadata_keys=["ссылка", "цена"]))

    return documents

df = pd.read_json("data/courses.jsonl", lines=True)
df.dropna(inplace=True)
df.drop_duplicates(subset=["title"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,url,title,price,description,technologies,has_none_values
0,https://gb.ru/geek_university/developer/archit...,DevOps-инженер с нуля до Middle,5223,"Вы научитесь настраивать пайплайны CI/CD, испо...","[Архитектура, DevOps-инженер, Ethernet, Unix, ...",False
1,https://gb.ru/geek_university/management/accou...,Профессия Бухгалтер,4049,"Бухгалтер следит за финансами компании, работа...","[Инструменты, 1С:Бухгалтерия 8, редакция 3.0, ...",False
2,https://gb.ru/geek_university/marketing/seo,Курс «SEO-специалист»,2550,Вы изучите основы маркетинга: от иследования р...,"[Яндекс.Метрика, Яндекс.Директ, Vk, Telegram, ...",False
3,https://gb.ru/geek_university/marketing/digita...,Маркетолог: быстрый старт в профессии,2121,"Станьте специалистом по продвижению товаров, у...","[SQL, HTML/CSS, Яндекс.Метрика, Яндекс.Директ,...",False
4,https://gb.ru/geek_university/developer/progra...,Курс «Разработчик искусственного интеллекта»,4771,"Создадите ИИ, которые распознают изображения и...",[Освоите 30+ популярных инструментов разработч...,False


In [4]:
docs = create_docs(df)

In [99]:
def setup_llm():
    system_prompt = "Ты AI ассистент, который на основе описания вакансии рекомендует курсы из контекста."
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )
    llm = HuggingFaceLLM(
        model_name="mistral_model",
        tokenizer_name="mistral_model",
        query_wrapper_prompt=PromptTemplate("<s>[INST] {query_str} [/INST] </s>\n"),
        context_window=8192,
        max_new_tokens=512,
        model_kwargs={"quantization_config": quantization_config},
        # tokenizer_kwargs={},
        generate_kwargs={"temperature": 0},
        device_map="auto",
        system_prompt=system_prompt,
    )
    return llm

In [100]:
llm = setup_llm()
Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(
    model_name="intfloat/multilingual-e5-base"
)

index = VectorStoreIndex.from_documents(documents=docs, show_progress=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/83 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/83 [00:00<?, ?it/s]

In [101]:
# qe_index = index.as_query_engine(similarity_top_k=7, node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.8)])
retriever = index.as_retriever(similarity_top_k=1, node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.1)])

In [146]:
stack_prompt = PromptTemplate(
    "У тебя есть описание вакансии: {query_str}\n"
    "Тебе нужно найти инструменты из описания вакансии и написать их на русском языке в виде строки с разделителем запятой и без повторений"
)

answer_prompt = PromptTemplate(
    "У тебя есть описание вакансии: {query_str}\n"
    "Тебе нужно найти инструменты из описания вакансии и написать их на русском языке в виде строки с разделителем запятой и без повторений"
)


class RAGStringQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: HuggingFaceLLM
    stack_prompt: PromptTemplate
    answer_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        query_str = re.sub("\n", " ", query_str)
        query_str = re.sub(" +", " ", query_str)
        query_str = query_str.strip()
        nodes = self.retriever.retrieve(query_str)
        stack = response = self.llm.complete(stack_prompt.format(query_str=query_str))
        responses = []
        for n in nodes:
            course = {
                "title": n.node.metadata["заголовок"],
                "price": n.node.metadata["цена"],
                "link": n.node.metadata["ссылка"],
                "desc": n.node.get_content()
            }
            response = self.llm.complete(
                stack_prompt.format(context_str=n.node.metadata["инструменты"], query_str=query_str)
            )
            print(str(response))
            course["recs"] = str(response).strip().split(", ")
            responses.append(course)
        
        return responses

In [147]:
synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RAGStringQueryEngine(
    retriever=retriever,
    response_synthesizer=synthesizer,
    llm=llm,
    stack_prompt=stack_prompt,
    answer_prompt=answer_prompt,
)

In [148]:
vacancy = """
Lamoda Group – это крупнейшая в России и СНГ онлайн-платформа по продаже fashion & lifestyle товаров.

В настоящий момент мы ищем коллегу на позицию Python/Go разработчика в команду Marketing Development, где ребята занимаются разными системами коммуникации с клиентами, а также множеством других сервисов, направленных на упрощение работы команды маркетинга :)


Готовы рассматривать кандидатов без опыта с Golang, но обязательно с желанием развиваться - у нас есть своя выстроенная система онбординга и обучения!

Как мы работаем:

пишем на последних версиях Python/Go;
используем NSQ для внутренних очередей, а Apache Kafka для межсервисного взаимодействия;
PostgreSQL в качестве основной СУБД, Redis, Aerospike;
все приложения упакованы в Docker и деплоятся нажатием кнопки;
у нас отличный CI/CD, запускаем тесты, security & style checks на каждый pull request;
все pull request проходят 360 code review;
участвуем в разработке и проектировании архитектурных решений, а также в реализации бизнес решений.
Чем предстоит заниматься:

писать новые и развивать существующие сервисы на Python и Go;
участвовать в анализе, проектировании и оценке технических решений вместе с командой;
писать unit и функциональные тесты;
участвовать в code review.
Мы ожидаем:

опыт коммерческой разработки бэкенда на Python от 2-х лет, опыт на Golang будет плюсом;
опыт работы с SQL базами данных (на уровне join-ов, оптимизации запросов);
умение проектировать API сервисов;
опыт работы с нереляционными базами данных (Redis);
опыт работы с Docker, k8s и Git.
"""

In [149]:
response = query_engine.query(vacancy)

/home/pc/Desktop/cp-cfo-2024/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


У тебя есть описание вакансии: Lamoda Group – это крупнейшая в России и СНГ онлайн-платформа по продаже fashion & lifestyle товаров. В настоящий момент мы ищем коллегу на позицию Python/Go разработчика в команду Marketing Development, где ребята занимаются разными системами коммуникации с клиентами, а также множеством других сервисов, направленных на упрощение работы команды маркетинга :) Готовы рассматривать кандидатов без опыта с Golang, но обязательно с желанием развиваться - у нас есть своя выстроенная система онбординга и обучения! Как мы работаем: пишем на последних версиях Python/Go; используем NSQ для внутренних очередей, а Apache Kafka для межсервисного взаимодействия; PostgreSQL в качестве основной СУБД, Redis, Aerospike; все приложения упакованы в Docker и деплоятся нажатием кнопки; у нас отличный CI/CD, запускаем тесты, security & style checks на каждый pull request; все pull request проходят 360 code review; участвуем в разработке и проектировании архитектурных решений, а 

In [150]:
print(response[0]["recs"])

['Python/Go разработчик', 'Marketing Development', 'NSQ', 'Apache Kafka', 'PostgreSQL', 'Redis', 'Aerospike', 'Docker', 'CI/CD', 'unit/functional tests', 'code review', 'SQL', 'API', 'non-relational databases', 'Docker', 'k8s', 'Git.']


In [11]:

# res = retriever.retrieve("На основе описания вакансии: \n" + vacancy + "\nПорекомендуй мне 1 курс из для каждого контекста и выведи их названия в виде списка")

In [12]:
# res

[NodeWithScore(node=TextNode(id_='0686d5af-8138-4623-9d97-29b581ecd75d', embedding=None, metadata={'ссылка': 'https://gb.ru/geek_university/developer/programmer/javascript/master', 'заголовок': 'Профессия Frontend-разработчик', 'цена': 4411, 'инструменты': 'Инструменты, VS Code, Vitest, Jest, Vite, Webpack, Pinia, Element Plus, Eslint, Sass, Pixel Perfect, Emmet, Lighthouse, HTML, CSS, GitLab, GitHub, Vue 3.0, React, TypeScript, JavaScript, REST API, Figma, Chrome DevTools, Навыки, Адаптивная верстка на HTML/CSS, Разработка интерактивных элементов на\xa0JavaScript, Работ с промисами, хранилищем данных и Cookie, Работа со сборщиками Webpack, Vite, Программирование на TypeScript, Работа с React.js/Vue.js, Командная разработка в Git, Работа с дизайн-макетами в Figma, Кросс-браузерная и адаптивная верстка, Оптимизация страниц и управление ресурсами, Написание тестов на Jest и Vitest'}, excluded_embed_metadata_keys=['ссылка', 'цена'], excluded_llm_metadata_keys=[], relationships={<NodeRelat